## Import library

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10940\1650128870.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Import utils

In [3]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

## Plotting sample

In [4]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='AAPL', start='2022-01-01', end='2024-12-31', freq='1d', test_size=0.5)
loader.run()
# test     

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


## Backtesting sample

In [5]:
strategy = EmaCross

grid_search = {
    'short_duration': range(2, 6),
    'long_duration': range(10, 16)
}

bt = BackTrader(data=loader.data, strategy=strategy, params=None, commission=0.002, grid_search=grid_search)

df = pd.DataFrame(bt.optimize()[1]).reset_index()
df.sort_values(by='Equity Final [$]', inplace=True, ascending=False)
df.head()

,short_duration,long_duration,Equity Final [$]
11,5,10,10151.370890
6,4,10,10121.676271
8,4,12,10083.106129
9,4,13,10081.741775
12,5,11,10080.003644


In [8]:
bt.optimize()[2]

 message: Maximum function evaluations reached (max_iter = 100)
 success: False
     fun: -10151.370890058608
       x: [5 10]
     nit: 10
    nfev: 100
      xv: [[5 10]
           [5 10]
           ...
           [5 14]
           [5 10]]
    funv: [-1.008e+04 -1.008e+04 ... -1.004e+04 -1.015e+04]
   space: <sambo._space.Space object at 0x0000019C86817C50>

In [24]:
bt.params = {
    'short_duration': 5,
    'long_duration': 10
}

bt.execute(mode='train')
bt.trades['train'].head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_EMA(C,5)","Exit_EMA(C,5)","Entry_EMA(C,10)","Exit_EMA(C,10)"
0,5,19,29,171.774277,168.243879,154.596849,188.951704,-17.651989,-0.020553,2022-01-31,2022-02-14,14 days,Long 0,164.810256,168.115231,164.190257,168.415472
1,5,30,32,170.034851,167.112177,153.031366,187.038336,-14.613371,-0.017189,2022-02-15,2022-02-17,2 days,Long 1,168.755104,168.131046,168.709904,168.413200
2,6,52,67,161.365311,166.019895,145.228780,177.501842,27.927506,0.028845,2022-03-18,2022-04-08,21 days,Long 2,157.572425,169.736248,157.273764,170.006384
3,6,102,109,146.681915,138.246029,132.013724,161.350107,-50.615315,-0.057511,2022-05-31,2022-06-09,9 days,Long 3,143.844407,143.878917,143.224148,144.183630
4,7,119,138,139.606033,153.566637,125.645430,153.566637,97.724223,0.100000,2022-06-24,2022-07-22,28 days,Long 4,135.780860,150.368814,135.614682,147.932752


In [25]:
# bt = BackTrader(data=loader.data, strategy=strategy, params=best, commission=0.002)
stats = bt.execute(mode='test')


In [26]:
bt.trades['test'].head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_EMA(C,5)","Exit_EMA(C,5)","Entry_EMA(C,10)","Exit_EMA(C,10)"
0,5,8,21,192.318848,183.921831,173.086963,211.550732,-41.985085,-0.043662,2023-07-17,2023-08-03,17 days,Long 0,189.810924,191.601489,189.361311,191.823459
1,5,38,45,178.878952,177.052337,160.991057,196.766847,-9.133077,-0.010211,2023-08-28,2023-09-07,10 days,Long 1,177.310744,181.765711,177.236980,181.796092
2,5,66,75,176.198593,174.034443,158.578734,193.818452,-10.820749,-0.012282,2023-10-06,2023-10-19,13 days,Long 2,173.597511,175.498896,173.100181,175.702361
3,5,85,110,176.278000,193.905800,158.650200,193.905800,88.139000,0.100000,2023-11-02,2023-12-08,36 days,Long 3,172.312582,192.387094,171.826592,190.925782
4,5,137,145,190.416763,182.891960,171.375087,209.458440,-37.624016,-0.039518,2024-01-19,2024-01-31,12 days,Long 4,186.516611,187.673659,185.865112,188.499679
